<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/U_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, concatenate
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

# Set your dataset path here
dataset_path = "/path/to/celeba/images"  # Update this path

# Load and preprocess the dataset
def load_images_from_folder(folder, num_images=10000):
    images = []
    for i, filename in enumerate(os.listdir(folder)):
        if i >= num_images:  # Limit the number of images to load for speed
            break
        img = load_img(os.path.join(folder, filename), target_size=(128, 128))
        img = img_to_array(img) / 255.0  # Normalize image to [0, 1]
        images.append(img)
    return np.array(images)

# Load images
images = load_images_from_folder(dataset_path)
x_train, x_test = train_test_split(images, test_size=0.1, random_state=42)

# Generate random masks for inpainting
def create_mask(shape):
    mask = np.ones(shape, dtype=np.float32)
    x, y, w, h = np.random.randint(30, 98), np.random.randint(30, 98), np.random.randint(30, 98), np.random.randint(30, 98)
    mask[x:x+w, y:y+h, :] = 0  # Place a zeroed-out box as a mask
    return mask

masks_train = np.array([create_mask((128, 128, 3)) for _ in range(len(x_train))])
masked_images_train = x_train * masks_train
masks_test = np.array([create_mask((128, 128, 3)) for _ in range(len(x_test))])
masked_images_test = x_test * masks_test

# Define U-Net model
def build_unet(input_shape=(128, 128, 3)):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    # Bottleneck
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)

    # Decoder
    u1 = UpSampling2D((2, 2))(c3)
    c4 = Conv2D(128, (3, 3), activation='relu', padding='same')(u1)
    merge1 = concatenate([c4, c2], axis=3)

    u2 = UpSampling2D((2, 2))(merge1)
    c5 = Conv2D(64, (3, 3), activation='relu', padding='same')(u2)
    merge2 = concatenate([c5, c1], axis=3)

    outputs = Conv2D(3, (1, 1), activation='sigmoid')(merge2)

    model = Model(inputs, outputs)
    return model

# Compile and train model
model = build_unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(masked_images_train, x_train, epochs=10, batch_size=32, validation_data=(masked_images_test, x_test))

# Display original, masked, and predicted images
def display_results(orig, masked, pred, num=5):
    plt.figure(figsize=(15, 5))
    for i in range(num):
        plt.subplot(3, num, i+1)
        plt.imshow(orig[i])
        plt.axis('off')
        plt.subplot(3, num, i+1+num)
        plt.imshow(masked[i])
        plt.axis('off')
        plt.subplot(3, num, i+1+num*2)
        plt.imshow(pred[i])
        plt.axis('off')
    plt.show()

# Predict and display results
predicted_images = model.predict(masked_images_test[:5])
display_results(x_test[:5], masked_images_test[:5], predicted_images)


FileNotFoundError: [Errno 2] No such file or directory: '/path/to/celeba/images'